In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt

# Distance matrix representing distances between cities
cities = [(0, 0), (1, 3), (4, 3), (6, 1), (7, 7), (8, 0), (3, 2)]

# Number of cities
n_cities = len(cities)

# Euclidean distance function between two cities
def euclidean_distance(city1, city2):
    return np.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

# Distance matrix
distances = np.zeros((n_cities, n_cities))
for i in range(n_cities):
    for j in range(n_cities):
        if i != j:
            distances[i][j] = euclidean_distance(cities[i], cities[j])

# Parameters for ACO
n_ants = 10  # Number of ants
n_iterations = 100  # Number of iterations
alpha = 1.0  # Influence of pheromone
beta = 2.0  # Influence of distance
rho = 0.1  # Pheromone evaporation rate
Q = 100  # Total pheromone to deposit

# Initialize pheromone matrix (all pheromones are the same at the beginning)
pheromone = np.ones((n_cities, n_cities)) / n_cities

# Function to construct a solution (tour) using a probabilistic approach
def construct_solution():
    # Randomly select the starting city
    start_city = random.randint(0, n_cities - 1)
    tour = [start_city]
    visited = [False] * n_cities
    visited[start_city] = True

    # Construct the rest of the tour
    for _ in range(n_cities - 1):
        current_city = tour[-1]
        probabilities = []
        for next_city in range(n_cities):
            if not visited[next_city]:
                # Calculate probability of selecting next_city
                pheromone_val = pheromone[current_city][next_city] ** alpha
                distance_val = distances[current_city][next_city] ** beta
                probabilities.append(pheromone_val / distance_val)
            else:
                probabilities.append(0)

        # Normalize probabilities
        total = sum(probabilities)
        probabilities = [p / total for p in probabilities]

        # Select the next city based on the probabilities
        next_city = np.random.choice(range(n_cities), p=probabilities)
        tour.append(next_city)
        visited[next_city] = True

    # Add the starting city to complete the tour (return to the origin)
    tour.append(start_city)
    return tour

# Function to update pheromones
def update_pheromones(ants_solutions, ants_distances):
    global pheromone
    # Evaporate pheromones
    pheromone *= (1 - rho)

    # Deposit new pheromones based on the ants' performances
    for solution, distance in zip(ants_solutions, ants_distances):
        for i in range(len(solution) - 1):
            city1 = solution[i]
            city2 = solution[i + 1]
            pheromone[city1][city2] += Q / distance
            pheromone[city2][city1] += Q / distance  # Symmetric TSP

# Function to run the ACO algorithm
def aco_tsp():
    best_tour = None
    best_distance = float('inf')

    # Run the algorithm for a number of iterations
    for _ in range(n_iterations):
        ants_solutions = []
        ants_distances = []

        # Each ant constructs a solution
        for _ in range(n_ants):
            solution = construct_solution()
            distance = sum(distances[solution[i]][solution[i + 1]] for i in range(len(solution) - 1))
            ants_solutions.append(solution)
            ants_distances.append(distance)

            # Update the best solution found
            if distance < best_distance:
                best_distance = distance
                best_tour = solution

        # Update the pheromone levels based on the solutions of the ants
        update_pheromones(ants_solutions, ants_distances)

    return best_tour, best_distance

# Run the Ant Colony Optimization for TSP
best_tour, best_distance = aco_tsp()

# Print the best solution found
print("Best tour:", best_tour)
print("Best distance:", best_distance)

# Plot the cities and the best route
def plot_tour(tour):
    tour_cities = [cities[i] for i in tour]
    tour_cities.append(tour_cities[0])  # Close the loop (return to the starting city)

    x, y = zip(*tour_cities)
    plt.plot(x, y, 'bo-', markersize=10)
    plt.title("Best Tour")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.show()

plot_tour(best_tour)

Best tour: [0, 3, 5, 4, 2, 6, 1, 0]
Best distance: 27.202457519704748
